In [1]:
import pandas as pd
import numpy as np
import math
pd.set_option('mode.chained_assignment',  None)

d1 = pd.read_csv('Indicators.csv')
d2 = pd.read_csv('Country.csv')

data = d1.copy()
df_2 = d2.copy()

# 처음 dataframe만들 때만 사용
# df : indicators.csv
# indicator : 사용할 indicator
# year : 선택할 year
def make_df(df,indicator,year):
    df_1 = df[df['Year'] == year]
    df_1 = df_1[df_1['IndicatorName'] == indicator]
    df_1 = df_1[['CountryCode','Value']]
    return df_1


# data : indicators.csv
# df : merge하기전 dataframe
# indicator : 새로 merge하고싶은 indicator
# year : mearge 하기 위한 year 선택
# df_2 : 새로 merge하기위해 만들어진 dataframe
def data_merge(df,indicator,year):
    df_2 = data[data['Year'] == year]
    df_2 = df_2[df_2['IndicatorName'] == indicator]
    df_2 = df_2[['CountryCode','Value']]
    df_merge = pd.merge(df, df_2, on='CountryCode', how='outer')
    #df_merge.columns = ['CountryCode', indicator+'_value',indicator2+'_value']
    #df_merge['Year'] = year
    return df_merge

#'Birth rate, crude (per 1,000 people)'
# Using indicators list
indi_list = ['Life expectancy at birth, total (years)','Age dependency ratio, old (% of working-age population)','Birth rate, crude (per 1,000 people)']
# 1st DataFrame
df1 = make_df(data,indi_list[0],2000)
# Merge 1 indicator
merge_1 = data_merge(df1,indi_list[1],2000)
# Merge 1 indicator 
merge_2 = data_merge(merge_1,indi_list[2],2000)

merge_2.columns = ['CountryCode',indi_list[0],indi_list[1],indi_list[2]]

merge_last = merge_2.copy()

# Check the Null value
def check_isnan_list(df,col):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    return null_list



# Check the Null value
def check_isnan_dict(df,col):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    null_dict = dict()
    for country in df['CountryCode'][null_list]:
        for index in range(len(df_2)):
            if df_2['CountryCode'][index] == country:
                region = df_2['Region'][index]
                null_dict[country] = region
    return null_dict

# df1 : Indicator.csv , df2 : Country.csv
def Check_country_IncomeGroup(df1,df2,group,indicator,year):
    country_group = df2[df2['IncomeGroup'] == group]['CountryCode']
    data_group = df1[df1['CountryCode'].isin(country_group)]
    return data_group[(data_group['IndicatorName'] == indicator) & (data_group['Year'] == year)]

# df1 : Indicator.csv , df2 : Country.csv
def Check_country_region(df1,df2,region,indicator,year):
    country_region = df2[df2['Region'] == region]['CountryCode']
    data_region = df1[df1['CountryCode'].isin(country_region)]
    return data_region[(data_region['IndicatorName'] == indicator) & (data_region['Year'] == year)]

# df1 : Indicator.csv , df2 : Country.csv
def Check_CurrencyUnit(df1,df2,currencyunit,indicator,year):
    unit = df2[df2['CurrencyUnit'] == currencyunit]['CountryCode']
    data_unit = df1[df1['CountryCode'].isin(unit)]
    return data_unit[(data_unit['IndicatorName'] == indicator) & (data_unit['Year'] == year)]

# Check the Null value
# df : merge가 끝난 dataframe
# col : null값을 채우기 위한 indicator value
# year : 해당 데이터셋의 year
# method : null값을 채우기 위한 기준
def change_null_value(df,col,year,method):
    null_list = []
    for i in range(len(df)):
        if math.isnan(df[col][i]):
            null_list.append(i)
        else:
            continue
            
    null_dict = dict()
    if method =='Region':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    region = df_2[method][index]
                    null_dict[country] = region
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_country_region(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            #dff = dff.T
            #dff.at[col,null_value] = new_value
            #dff = dff.T
            df[col][null_value] = new_value
        return df
    
    elif method =='IncomeGroup':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    group = df_2[method][index]
                    null_dict[country] = group
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_country_IncomeGroup(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            df[col][null_value] = new_value       
        return df
    
    elif method == 'CurrencyUnit':
        for country in df['CountryCode'][null_list]:
            for index in range(len(df_2)):
                if df_2['CountryCode'][index] == country:
                    unit = df_2[method][index]
                    null_dict[country] = unit
        keys = list(null_dict.keys())        
        for key,null_value in zip(keys,null_list):
            new_value_data = Check_CurrencyUnit(data,df_2,null_dict[key],col,year)
            new_value = np.average(new_value_data['Value'])
            df[col][null_value] = new_value       
        return df        

In [2]:
df_region = merge_last.copy()
df_region = change_null_value(df_region,indi_list[0],2000,'Region')
df_region = change_null_value(df_region,indi_list[1],2000,'Region')
df_region = change_null_value(df_region,indi_list[2],2000,'Region')

In [3]:
df_group = merge_last.copy()
df_group = change_null_value(df_group,indi_list[0],2000,'IncomeGroup')
df_group = change_null_value(df_group,indi_list[1],2000,'IncomeGroup')
df_group = change_null_value(df_group,indi_list[2],2000,'IncomeGroup')

In [4]:
df_unit = merge_last.copy()
df_unit = change_null_value(df_unit,indi_list[0],2000,'CurrencyUnit')
df_unit = change_null_value(df_unit,indi_list[1],2000,'CurrencyUnit')
df_unit = change_null_value(df_unit,indi_list[2],2000,'CurrencyUnit')

C:\Users\leejy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\leejy\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
df_region

,CountryCode,"Life expectancy at birth, total (years)","Age dependency ratio, old (% of working-age population)","Birth rate, crude (per 1,000 people)"
0,ARB,67.826949,7.041462,28.282837
1,CSS,69.438262,11.075336,20.044584
2,CEB,72.719690,19.864752,9.770916
3,EAS,71.662834,10.336070,15.796701
4,EAP,70.690575,9.158629,16.400833
5,EMU,78.273992,24.051069,10.637990
6,ECS,73.026072,19.969235,11.668391
7,ECA,68.895651,14.370703,15.354005
8,EUU,77.157631,23.304327,10.593453
9,FCS,57.040161,6.265157,38.058768


In [6]:
df_group

,CountryCode,"Life expectancy at birth, total (years)","Age dependency ratio, old (% of working-age population)","Birth rate, crude (per 1,000 people)"
0,ARB,67.826949,7.041462,28.282837
1,CSS,69.438262,11.075336,20.044584
2,CEB,72.719690,19.864752,9.770916
3,EAS,71.662834,10.336070,15.796701
4,EAP,70.690575,9.158629,16.400833
5,EMU,78.273992,24.051069,10.637990
6,ECS,73.026072,19.969235,11.668391
7,ECA,68.895651,14.370703,15.354005
8,EUU,77.157631,23.304327,10.593453
9,FCS,57.040161,6.265157,38.058768


In [7]:
df_unit

,CountryCode,"Life expectancy at birth, total (years)","Age dependency ratio, old (% of working-age population)","Birth rate, crude (per 1,000 people)"
0,ARB,67.826949,7.041462,28.282837
1,CSS,69.438262,11.075336,20.044584
2,CEB,72.719690,19.864752,9.770916
3,EAS,71.662834,10.336070,15.796701
4,EAP,70.690575,9.158629,16.400833
5,EMU,78.273992,24.051069,10.637990
6,ECS,73.026072,19.969235,11.668391
7,ECA,68.895651,14.370703,15.354005
8,EUU,77.157631,23.304327,10.593453
9,FCS,57.040161,6.265157,38.058768
